In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
tf.__version__

# Classification

In [ ]:
fmnist_data = keras.datasets.fashion_mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = fmnist_data.load_data()

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_train[:10]

In [ ]:
plt.imshow(x_train[0,], cmap="gray")

In [ ]:
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
x_train.shape

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(75, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

# Define Model 

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(75, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
weights, bias = model.layers[1].get_weights()

In [ ]:
weights

In [ ]:
bias

# Compile Model

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

# Fit Model

In [ ]:
history = model.fit(x_train, y_train, epochs=50, validation_split=0.15)

In [ ]:
print(history.history)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(history.history["loss"], label="train loss")
ax.plot(history.history["accuracy"], label="train accuracy")
ax.plot(history.history["val_loss"], label="validation loss")
ax.plot(history.history["val_accuracy"], label="validation accuracy")
ax.legend()

In [ ]:
model.evaluate(x_test, y_test, verbose=0)

In [ ]:
x3 = x_test[1:3,:,:]
x3.shape

In [ ]:
plt.imshow(x3[0], cmap="gray")

In [ ]:
model.predict(x3).round(3)

In [ ]:
model.predict(x3).shape

In [ ]:
np.argmax(model.predict(x3).round(3), axis=1)

In [ ]:
y_test[1:3]

# Regression

In [ ]:
california_housing = fetch_california_housing()
california_housing

In [ ]:
X_train0, X_test, y_train0, y_test = train_test_split(
                 california_housing["data"],
                 california_housing["target"])

In [ ]:
X_train1, X_validation, y_train1, y_validation = train_test_split(X_train0,
                                                                  y_train0)

In [ ]:
X_test.shape

In [ ]:
X_train0.shape

In [ ]:
X_train1.shape

In [ ]:
X_validation.shape

In [ ]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.transform(X_validation)
X_test_s = sc.transform(X_test)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1)
])

In [ ]:
model.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
model.fit(X_train_s, y_train1, epochs=30,
          validation_data=(X_validation_s, y_validation))

# Functional API

In [ ]:
X_train0.shape[1:]

In [ ]:
input_ = keras.layers.Input(shape=X_train1.shape[1:])
hidden_layer1 = keras.layers.Dense(50, activation="relu")(input_)
hidden_layer2 = keras.layers.Dense(10, activation="relu")(hidden_layer1)
concatenate_layer = keras.layers.Concatenate()([input_, hidden_layer2])
output = keras.layers.Dense(1)(concatenate_layer)
model = keras.Model(inputs=[input_], outputs=[output])

In [ ]:
model.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
model.fit(X_train_s, y_train1, epochs=30,
          validation_data=(X_validation_s, y_validation))

In [ ]:
X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, -4:]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, -4:]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, -4:]

In [ ]:
input_1 = keras.layers.Input(shape=[6])
input_2 = keras.layers.Input(shape=[4])
hidden_layer1 = keras.layers.Dense(50, activation="relu")(input_1)
hidden_layer2 = keras.layers.Dense(10, activation="relu")(hidden_layer1)
concatenate_layer = keras.layers.Concatenate()([input_2, hidden_layer2])
output = keras.layers.Dense(1)(concatenate_layer)
model = keras.Model(inputs=[input_1, input_2], outputs=[output])

In [ ]:
model.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
model.fit((X_train_s_1, X_train_s_2), y_train1, epochs=30,
          validation_data=((X_validation_s_1, X_validation_s_2),
                           y_validation))

In [ ]:
input_1 = keras.layers.Input(shape=[6])
input_2 = keras.layers.Input(shape=[4])
hidden_layer1 = keras.layers.Dense(50, activation="relu")(input_1)
hidden_layer2 = keras.layers.Dense(10, activation="relu")(hidden_layer1)
concatenate_layer = keras.layers.Concatenate()([input_2, hidden_layer2])
output = keras.layers.Dense(1, name="output")(concatenate_layer)
helper_output = keras.layers.Dense(1, name="helper_output")(hidden_layer2)
model_f = keras.Model(inputs=[input_1, input_2],
                    outputs=[output, helper_output])

In [ ]:
model_f.compile(loss=["mse", "mse"],
              loss_weights = [0.8, 0.2],
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
model_f.fit((X_train_s_1, X_train_s_2), (y_train1, y_train1), epochs=30,
          validation_data=((X_validation_s_1, X_validation_s_2),
                           (y_validation, y_validation)))

# Subclass API

In [ ]:
class WideAndDeepANN(keras.Model):
    def __init__(self, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer_1 = keras.layers.Dense(50, activation=activation)
        self.hidden_layer_2 = keras.layers.Dense(10, activation=activation)
        self.final_output   = keras.layers.Dense(1)
        self.helper_output  = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_1, input_2 = inputs
        h1_out = self.hidden_layer_1(input_1)
        h2_out = self.hidden_layer_2(h1_out)
        concat_out = keras.layers.concatenate([input_2, h2_out])
        final_out = self.final_output(concat_out)
        helper_out = self.helper_output(h2_out)
        return final_out, helper_out

In [ ]:
model_sub = WideAndDeepANN()

In [ ]:
model_sub.compile(loss=["mse", "mse"],
              loss_weights = [0.8, 0.2],
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [ ]:
model_sub.fit((X_train_s_1, X_train_s_2), (y_train1, y_train1), epochs=30,
          validation_data=((X_validation_s_1, X_validation_s_2),
                           (y_validation, y_validation)))

In [ ]:
model_f.summary()

In [ ]:
model_f.save("housing_reg_model_f.h5")

In [ ]:
model_f_reg = keras.models.load_model("housing_reg_model_f.h5")

In [ ]:
model_f_reg.summary()

In [ ]:
model_sub.summary()

In [ ]:
model_sub.save("housing_reg_model_sub.h5")
#.save_weight()

# Callback

In [ ]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint("model_cb_reg_housing.h5",
                                                         save_best_only=True)
earlystopping_callback = keras.callbacks.EarlyStopping(patience=5,
                             restore_best_weights=True)

In [ ]:
model_f.fit((X_train_s_1, X_train_s_2), (y_train1, y_train1), epochs=5,
          validation_data=((X_validation_s_1, X_validation_s_2),
                           (y_validation, y_validation)),
           callbacks=[model_checkpoint_callback, earlystopping_callback])

In [ ]:
class MyCallback(keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        print("")
    def on_epoch_end(self, epoch, logs):
        print(logs["val_loss"])

In [ ]:
mycb = MyCallback()

In [ ]:
tb_callback = keras.callbacks.TensorBoard("tb_log")

In [ ]:
model_f.fit((X_train_s_1, X_train_s_2), (y_train1, y_train1), epochs=5,
          validation_data=((X_validation_s_1, X_validation_s_2),
                           (y_validation, y_validation)),
           callbacks=[tb_callback])

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=./tb_log 

# Hyperparameter optimization

In [ ]:
def ann_model(number_of_hidden_layers=1,
              number_of_neurons=50,
              lr=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[8]))
    for hidden_layer in range(number_of_hidden_layers):
        model.add(keras.layers.Dense(number_of_neurons, activation="selu"))
    model.add(keras.layers.Dense(1))
    sgd = keras.optimizers.SGD(lr=lr)
    model.compile(loss="mse", optimizer=sgd)
    return model

In [ ]:
keras_sk_reg = keras.wrappers.scikit_learn.KerasRegressor(build_fn=ann_model)

In [ ]:
param_grid = {"number_of_hidden_layers":[1, 3, 5],
              "number_of_neurons":[50, 100, 150],
              "lr":[0.001, 0.001, 0.1]}

In [ ]:
keras_sk_reg_gs = GridSearchCV(keras_sk_reg, param_grid)

In [ ]:
keras_sk_reg_gs.fit(X_train_s, y_train1, epochs=10,
                    validation_data=(X_validation, y_validation),
                    callbacks=[keras.callbacks.EarlyStopping(patience=5)])